In [ ]:
import torch
from torch import nn, optim
from torchvision import models, transforms,datasets
from PIL import Image
import mltools

In [ ]:
def tfrs(transform, path="../data", batch_size=8, num_workers=8, pin_memory=True, drop_last=True):
    data = datasets.ImageFolder(f"{path}/TFRS", transform=transform)
    train_data, val_data, test_data = mltools.split_data(data, [0.7, 0.15, 0.15])
    return mltools.iter_data([train_data, val_data, test_data], batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last)

In [ ]:
device = torch.device("cuda")
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor(), transforms.ConvertImageDtype(torch.float32), transforms.Normalize(mean=0.5, std=0.5)])
train_iter, val_iter, test_iter = tfrs(transform)  # 获取训练集、验证集、测试集
labels = train_iter.dataset.dataset.classes
model = models.resnet18(num_classes=len(labels))  # 设置模型结构
model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
ml = mltools.MachineLearning("TFRS")
ml.add_model(model)
epoch, timer, recorder = ml.batch_create()

In [ ]:
# 训练
num_epochs = epoch(3)
animator = ml.create_animator(xlabel="epoch", xlim=[0, epoch.totol_epoch + 1], ylim=-0.1, legend=["train loss", "train acc", "val acc"])  # 创建动画器
for current_epoch in range(1, num_epochs + 1):
    timer.start()  # 开始计时

    # 计算训练集
    metric_train = mltools.Accumulator(3)  # 累加器：(train_loss, train_acc, train_size)
    model.train()  # 训练模式
    for x, y in train_iter:
        x, y = x.to(device), y.to(device)  # 转换x、y
        y_train = model(x)  # 计算模型
        train_loss = loss(y_train, y)  # 计算损失
        train_pred = y_train.argmax(dim=1)  # 计算预测值
        train_acc = (train_pred == y).sum()  # 计算准确数

        # 梯度更新
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        metric_train.add(train_loss * y.numel(), train_acc, y.numel())  # 累加损失、准确数、样本数
    recorder[0].append(metric_train[0] / metric_train[2])  # 计算平均损失
    recorder[1].append(metric_train[1] / metric_train[2])  # 计算准确率

    # 计算验证集
    metric_val = mltools.Accumulator(2)  # 累加器：(val_acc, val_size)
    model.eval()  # 验证模式
    with torch.no_grad():
        for x, y in val_iter:
            x, y = x.to(device), y.to(device)  # 转换x、y
            y_val = model(x)  # 计算模型
            val_acc = (y_val.argmax(dim=1) == y).sum()  # 计算准确数
            metric_val.add(val_acc, y.numel())  # 累加准确数、样本数
    recorder[2].append(metric_val[0] / metric_val[1])  # 计算准确率

    timer.stop()  # 停止计时

    # 打印输出值
    ml.logger.info(f"train loss {recorder[0][-1]:.3f}, train acc {recorder[1][-1]:.3f}, val acc {recorder[2][-1]:.3f}")
    ml.print_training_time_massage(timer, num_epochs, current_epoch)
    ml.logger.info(f"trained on {str(device)}")
    animator.show(recorder.data)
else:
    # 打印输出值
    ml.logger.info(f"train loss {recorder[0][-1]:.3f}, train acc {recorder[1][-1]:.3f}, val acc {recorder[2][-1]:.3f}")
    ml.print_training_time_massage(timer, num_epochs, current_epoch)
    ml.logger.info(f"trained on {str(device)}")
    animator.show(recorder.data)
ml.save()

In [ ]:
# 测试
model.eval()
metric = mltools.Accumulator(2)  # 累加器：(test_acc, test_size)
with torch.no_grad():
    for x, y in test_iter:
        x, y = x.to(device), y.to(device)  # 转换x、y
        y_test = model(x)  # 计算模型
        test_pred = y_test.argmax(dim=1)  # 计算预测值
        test_acc = (test_pred == y).sum()  # 计算准确数
        metric.add(test_acc, y.numel())  # 累加准确数、样本数
ml.logger.info(f"test acc {metric[0] / metric[1]:.3f}")  # 计算准确率并输出

In [ ]:
# 预测
image = "../data/TFRS/仿宋/仿宋_01.png"
model.eval()
with torch.no_grad():
    image_tensor = transform(Image.open(image).convert("RGB")).unsqueeze(0)
    image_tensor = image_tensor.to(device)
    print(labels[model(image_tensor).argmax(dim=1)[0]])